In [ ]:
# First approach to Smart Charging Phase 1 using PuLP.
# Modeled as a PuLP optimisation blending problem
# Started 20 Aug 2020
# Author: Sofia Taylor, Flexible Power Systems

import numpy as np
import pandas as pd
import datetime as dt
from pulp import *
import pickle
import global_variables as gv
import output_functions as of
import matplotlib as mpl
import matplotlib.pyplot as plt
import time
import random
import matplotlib.dates as mdates
import matplotlib.font_manager as font_manager
import lin_prog_functions as lpf
import testdata_proc as pf
import plotly.graph_objects as go
import plotly.io as pio

In [ ]:
# grid_file_path = 'Outputs/Logs/grid_variables{}.csv'.format(run)
journeys_range = pickle.load(open('Outputs/journeys_range','rb'))
price_data = pickle.load(open('Outputs/price_data','rb'))
empty_profile = pickle.load(open('Outputs/empty_profile','rb'))
chargers = [22,22]
charger = 22
capacity = 40
ca='opt'
site_capacity = {
    'opt': capacity,  
    'BAU': 10000,
    'BAU2': capacity
}

In [ ]:
empty_profile = empty_profile.loc[(slice(None), 3), :]

In [ ]:
## Single Day test
req_energy = empty_profile.groupby(
    ['date','Vehicle_ID']).sum()[['Battery_Use']]*(
        1+gv.MARGIN_SOC)
last_day = req_energy.index[-1][0]+dt.timedelta(days=1)
for v in empty_profile.index.get_level_values('Vehicle_ID').unique():
    req_energy.loc[(last_day,v),'Battery_Use'] = 0
battery_cap = gv.VSPEC['Vivaro_LR']['C']
req_energy['Full_Use'] = -battery_cap
req_energy['Req_Battery'] = req_energy[['Battery_Use','Full_Use']].max(axis=1)
day = dt.datetime(2019,4,29)
day_profile = pf.create_daily_schedule(empty_profile, day)
next_req = req_energy.loc[(day+dt.timedelta(days=1),slice(None)),'Req_Battery'].droplevel(level=0)
initial_rel_charge = pd.Series(
        data = [0],  # *gv.NUM_VEHICLES,
        index = empty_profile.index.get_level_values(1).unique()
    )
output_df, PuLP_prob, fsoc, _, opt_level = lpf.linear_optimiser_V4(
                    day_profile,
                    ca,
                    chargers[0],chargers[1],
                    capacity,
                    initial_rel_charge,
                    next_req, battery_cap
                    )

day_profile_out = day_profile.merge(
                output_df,
                how='left',
                left_index=True,
                right_index=True,
                )
range_profile = day_profile_out.fillna(0)
cols=gv.CAT_COLS
vehicles = output_df.index.get_level_values(1).unique()

range_profile[cols['CHARGE_DEL'][ca]] = (
    range_profile[cols['OUTPUT'][ca]] 
    * gv.CHARGER_EFF)
range_profile[cols['ECOST'][ca]] = (
    range_profile[cols['OUTPUT'][ca]] 
    * range_profile[cols['PRICE']['opt']])
for vehicle in vehicles:
    range_profile.loc[(slice(None),vehicle),cols['SOC'][ca]] = (
        battery_cap + initial_rel_charge
        + range_profile.loc[(slice(None),vehicle),cols['CHARGE_DEL'][ca]].cumsum() 
        + range_profile.loc[(slice(None),vehicle),'Battery_Use'].cumsum()
        )*100/battery_cap

In [ ]:
fig, axs = plt.subplots(
    4,
    figsize=(12,10),
    sharex=True, 
    gridspec_kw={'hspace':0.1})
ca='opt'
x = range_profile.unstack().index.strftime('%H:%M')
cols = gv.CAT_COLS

axs[0].plot(
    x, 
    range_profile.unstack()[cols['OUTPUT'][ca]]*2)
axs[0].legend(range_profile.unstack()[cols['OUTPUT'][ca]].columns)
# axs[0].plot(
#     x,
#     range_profile[cols['OUTPUT'][ca]].groupby('from').sum()*2, linewidth=3)
axs[0].set_ylabel('Demand (kW)')

axs[1].plot(
    x,
    range_profile.unstack()[cols['ECOST'][ca]]/100)
# axs[1].plot(
#     x,
#     range_profile[cols['ECOST'][ca]].groupby('from').sum()/100, linewidth=3)
axs[1].set_ylabel('Electricity Costs (GBP)')

axs[2].plot(
    x, 
    range_profile.unstack()[cols['SOC'][ca]])
axs[2].plot(
    x,
    range_profile[cols['SOC'][ca]].groupby('from').mean(), linewidth=3)
axs[2].set_ylabel('SOC(%)')

axs[3].plot(
    x,
    range_profile.unstack()[cols['PRICE']['opt']], 
    color=gv.FPS_PURPLE)
axs[3].set_ylabel('Electricity Price (p/kWh)')
for ax in fig.get_axes():
    ax.xaxis.set_major_locator(plt.MaxNLocator(24))
fig.show()

# print((range_profile[cols['OUTPUT'][ca]].groupby('from').sum()>capacity/2).sum())
# grouped = range_profile[cols['OUTPUT'][ca]].groupby('from').sum()
# grouped[grouped>capacity/2]

In [ ]:
range_profile.sort_values(by='SoC_Opt').head()

In [ ]:
# ## Range Test

# run = 130
# notes = 'Test'

# script_strt = time.process_time()
# print('Run:',run,'/ Charger:',chargers[0],chargers[1],'/ Capacity:',capacity)
# os.makedirs('Outputs/Logs/run{}'.format(run))

# profile_out, dates, bad_days, lpprob, status = lpf.optimise_range2(
#     empty_profile, 
#     chargers, 
#     site_capacity)

# range_profile, site_profile, days_summary, global_summary = of.summary_outputs(
#     profile_out, 
#     journeys_range,     capacity, status)

# os.makedirs('Outputs/Logs/run{}/daily'.format(run))
# for date in dates:
#     day = dt.datetime.combine(date, dt.datetime.min.time())
#     day_profile = of.create_daily_summary(site_profile, day)
#     fig_summary = of.summary_plot(day_profile)
#     fig_summary.savefig(
#         'Outputs/Logs/run{}/daily/fig{}.jpg'.format(run,date))
#     plt.close(fig_summary)

# with open('Outputs/Logs/run{}/variables{}.csv'.format(run,run),'a') as fi:
#     fi.write(notes)
#     fi.write('\n' + str(run)+','+str(charger) + ',' + str(capacity) +'\n')
#     fi.write(global_summary.to_string())
#     fi.write(bad_days)

# range_fig = of.daily_summary_plot(days_summary.fillna(0))
# range_fig.savefig(
#             'Outputs/Logs/run{}/fig_range{}.svg'.format(run,run),
#             bbox_inches = "tight")
# range_fig.show()